## Database & Jupyter Notebook Setup
---

import the dataset: 
`mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray`

In [1]:
# import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# create an instance of MongoClient
mongo = MongoClient(port=27017)

# confirm db was created in MongoClient
mongo.uk_food

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food')

In [ ]:
# create db variable
db = mongo['uk_food']

# list collections in db
filter = {"name": {"$regex": r"^(?!system\.)"}}
db.list_collection_names(filter=filter)

['establishments']

In [16]:
# create collection variable
establishments = db['establishments']

# review a document in the collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('677c79000e404675433df383'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0